In [2]:
import requests
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import BaseTool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [12]:
llm = ChatGoogleGenerativeAI(
    google_api_key=os.getenv("GOOGLE_API_KY"),
    model="gemini-pro",
)

In [13]:
class WeatherTemperatureTool(BaseTool):
    name = "weather_tool"
    description = "The tool useful for get temperature based on location"

    def get_weather_temperature(self, location: str):
        url = f"https://api.tomorrow.io/v4/weather/realtime?location={location}&apikey={os.getenv('TOMORROW_API_KEY')}"
        headers = {"accept": "application/json"}
        response = requests.get(url, headers=headers)
        json = response.json()
        return json["data"]["values"]

    def _run(self, query: str):
        weather_data = self.get_weather_temperature(query)
        return weather_data["temperature"]

    def _arun(self):
        raise NotImplementedError("This tool does not support async")


In [14]:
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
]

In [15]:
prompt = ChatPromptTemplate.from_messages(messages)

In [16]:
tools = [WeatherTemperatureTool()]

In [17]:
agent = create_openai_functions_agent(
    llm=llm, tools=tools, prompt=prompt)

In [18]:
agent_executer = AgentExecutor(agent=agent, tools=tools)

In [19]:
agent_executer.invoke({"input": 'tehran temperature'})

{'input': 'tehran temperature',
 'output': 'It is currently 14.19 degrees Celsius in tehran.'}